In [2]:
# 필요한 패키지 불러오기
import pandas as pd
from konlpy.tag import Okt 
okt = Okt()
import gensim
import gensim.corpora as corpora
import os
# os.environ["PYTHONIOENCODING"] = "utf-8”
import pyLDAvis.gensim

In [17]:
# 저장한 데이터 불러오기
df = pd.read_excel('(1)samsung_all_buds.xlsx')
df.head(20)

,ratings,reviews,label
0,5,"가성비 갑, 성능 좋아요!! 추천드립니다! **내돈내산 후기입니다**구매 및 개봉일...",1
1,5,전직 컴수리기사의 버즈fe 테스트 결과보고서 주 문 : 24. 3. 27도 착 :...,1
2,5,가성비 그자체의 무선 이어폰입니다 내돈내산 솔직후기!!!---------------...,1
3,5,러닝용으로 구매했는데 절대 안빠지고 잘 버텨주는 제품 작년 겨울에 쿠팡에서 산 저렴...,1
4,5,"""화이트 갤럭시 버즈 FE, 내돈내산! 최고! (づ｡◕‿‿◕｡)づ삼성전자 갤럭시 버...",1
5,5,가성비 굿 신상 최고 안녕하세요제가 버즈시리즈는 전부 가지고 있는 사람입니다!버즈 ...,1
6,5,가성비 커널형 이어폰 ✅️ 갤럭시 버즈 FE 사용하는 이유:✅️1. 편리한 무선 사...,1
7,5,가격대비 성능 좋은 보급형 무선이어폰 #내돈내산후기#버즈 2를 사용하다가 한쪽 유닛...,1
8,5,등록된 헤드라인이 없습니다 늦깎이 대학생인 남편 위해 구매했어요-----------...,1
9,5,[가성비의 끝판왕] 삼성전자 갤럭시 버즈 FE 후기 [가성비의 끝판왕] 삼성전자 갤...,1


In [4]:
# 리뷰 데이터만 가져오기
review = df['reviews']

# 빈 리스트를 만들어 명사 토큰을 추출해 저장한다.
review_noun_tk = []
for n in review:
    review_noun_tk.append(okt.nouns(n)) # 명사 형태소만 출력

In [5]:
#한글자로만 추출된 데이터를 제거하기
review_noun_tk2 = []
for d in review_noun_tk:
    item = [i for i in d if len(i) > 1] # 뽑아낸 토큰 중 크기가 1 이상만 추출
    review_noun_tk2.append(item)

## LDA 모델 구축

In [6]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

In [7]:
# '등록 헤드라인 등록 리뷰 용이' 필요없는 명사 삭제처리
removeWords = ['등록', '헤드라인', '리뷰', '용이', '제품']

ids_to_remove = [dict.token2id[word] for word in removeWords if word in dict.token2id]
dict.filter_tokens(bad_ids=ids_to_remove)

In [8]:
# 딕셔너리화 성공~
print(dict.token2id)

{'가격': 0, '가성': 1, '개봉': 2, '개봉일': 3, '개인': 4, '갤럭시': 5, '경우': 6, '고무': 7, '고민': 8, '과소': 9, '구매': 10, '귀가': 11, '그레이': 12, '기능': 13, '기존': 14, '기차': 15, '내부': 16, '노이즈': 17, '다만': 18, '다음': 19, '더욱': 20, '도움': 21, '도착': 22, '뒤쪽': 23, '등등': 24, '때문': 25, '말소리': 26, '모두': 27, '방송': 28, '배터리': 29, '버드': 30, '버즈': 31, '버튼': 32, '변경': 33, '별로': 34, '보급': 35, '보조': 36, '본체': 37, '부분': 38, '분실': 39, '브랜드': 40, '블랙': 41, '블루투스': 42, '비일': 43, '뽁뽁이': 44, '사고': 45, '사람': 46, '사용': 47, '사용자': 48, '사유': 49, '사진': 50, '상자': 51, '생각': 52, '선택': 53, '설명': 54, '성능': 55, '성품': 56, '슬링': 57, '시간': 58, '실제': 59, '쓰기': 60, '양쪽': 61, '어댑터': 62, '여행': 63, '연한': 64, '오히려': 65, '완전': 66, '우선': 67, '위아래': 68, '음질': 69, '이어폰': 70, '인식': 71, '일단': 72, '일이': 73, '자주': 74, '전용': 75, '정도': 76, '정말': 77, '제거': 78, '제외': 79, '준수': 80, '직접': 81, '차안': 82, '차이': 83, '차이나': 84, '착용': 85, '첨부': 86, '추천': 87, '출발': 88, '충전': 89, '측면': 90, '컴퓨터': 91, '케이스': 92, '타입': 93, '택배': 94, '패킹': 95, '편이': 96, '한쪽': 97, '합리': 98, '

In [9]:
# 단어의 빈도수를 확인하기 위해 doc2bow 로 변환한다.
# BoW 모델은 문서의 단어의 빈도수를 표현해준다.
corpus = [dict.doc2bow(word) for word in review_noun_tk2]

In [10]:
# corpus 출력을 통해 (word_id, work_count)의 BoW 구성을 확인
print(corpus)

[[(0, 3), (1, 3), (2, 2), (3, 1), (4, 1), (5, 4), (6, 2), (7, 2), (8, 2), (9, 1), (10, 4), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 11), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 4), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 4), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 5), (56, 1), (57, 2), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 2), (71, 2), (72, 1), (73, 1), (74, 1), (75, 2), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 2), (82, 1), (83, 2), (84, 1), (85, 1), (86, 1), (87, 2), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 3), (94, 1), (95, 2), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 3), (103, 1)], [(0, 4), (1, 3), (5, 4), (11, 3), (21, 2), (22, 1), (24, 2), (27, 

In [14]:
# 토픽의 갯수 k 개 생성
k = 4

#gensim 라이브러리를 사용해 lda 모델을 생성하는 코드 각 옵션은 만들어놓은 
#corpus lda모델
#iterations 각 문서의 주제 할당을 반복하는 횟수
#num_topics 는 위에 지정한 k 값으로 토픽의 갯수
#id2word 각 토큰의 id에 해당하는 토큰을 연결해준다
#passes는 앞서 넣은 corpus를 학습 반복하는 횟수이다 수가 높아지면 더 잘 학습하지만 그만큼 시간이 증가한다.
#workers 쓰레드의 수 (작업자의 수)이다. 보통은 본인의 컴퓨터에 맞춰 설정하게 된다.
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations = 12, num_topics = k, id2word = dict, passes = 5, workers = 10)

In [15]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.054*"버즈" + 0.023*"사용" + 0.016*"구매" + 0.016*"착용" + 0.015*"이어폰" + 0.015*"음질" + 0.014*"라이브" + 0.012*"가격" + 0.012*"갤럭시" + 0.011*"선물" + 0.010*"고민" + 0.009*"소리" + 0.009*"케이스" + 0.008*"삼성" + 0.008*"프로"'), (1, '0.036*"이어폰" + 0.029*"버즈" + 0.020*"사용" + 0.019*"갤럭시" + 0.017*"음질" + 0.016*"에어팟" + 0.016*"구매" + 0.014*"프로" + 0.011*"통화" + 0.010*"기능" + 0.009*"착용" + 0.009*"생각" + 0.008*"가격" + 0.008*"삼성" + 0.007*"무선"'), (2, '0.062*"버즈" + 0.029*"프로" + 0.026*"음질" + 0.023*"노이즈" + 0.021*"슬링" + 0.021*"소리" + 0.019*"사용" + 0.017*"이어폰" + 0.016*"구매" + 0.016*"착용" + 0.011*"플러스" + 0.011*"기능" + 0.010*"가격" + 0.010*"에어팟" + 0.008*"생각"'), (3, '0.025*"음질" + 0.023*"사용" + 0.022*"버즈" + 0.020*"착용" + 0.014*"이어폰" + 0.012*"소리" + 0.012*"디자인" + 0.011*"배송" + 0.011*"구매" + 0.010*"가격" + 0.010*"통화" + 0.009*"충전" + 0.008*"갤럭시" + 0.008*"정도" + 0.007*"삼성"')]


In [16]:
# lda모델을 디스플레이로 출력
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict)
pyLDAvis.display(lda_vis)

### 토픽분석을 통해 얻을 수 있는 정보
1. 삼성에서는 애플에 에어팟에 비교하는 댓글들이 많음
2. 애플의 에어팟에 비하여 가격이 저렴하기에 구매하는 경우가 많음
3. 주로 가격이 크게 비싸지 않기에 다른 사람에게 선물을 한다는 리뷰가 많음